# readme
descrip: resample CMEMS (AVISO) daily ADT and Vgeo to monthly mean and save the data to nc files for future plot

update history: <br>
v1.0 DL 2021Mar30 <br>
v1.1 DL 2021Apr05 <br>
v1.2 DL 2021Apr24 <br>
v1.3 DL 2021May18

extra notes: <br>

# import modules

In [1]:
# general python packages
import numpy as np
import xarray as xr
# import pandas as pd
import matplotlib.pyplot as plt
# import matplotlib.dates as mdates
%matplotlib inline
# import cartopy.crs as ccrs
# from cartopy.mpl.geoaxes import GeoAxes
# from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
# from mpl_toolkits.axes_grid1 import AxesGrid

# python GCM(POP) packages
# import xgcm
# import pop_tools
# import xesmf as xe

# from eofs.xarray import Eof

# dask jupyter lab packages
from dask.distributed import Client
# from dask.distributed import performance_report

# file name with time packages
from itertools import product
# from cftime import DatetimeNoLeap

# incorporate dask 

In [2]:
client = Client("tcp://10.73.2.193:37188")
client

Client Scheduler: tcp://10.73.2.193:37188 Dashboard: http://10.73.2.193:8787/status,Cluster Workers: 16 Cores: 16 Memory: 350.08 GB


In [3]:
date_str = '2021May18'

yrs = np.arange(1993,2019+1)
mons = np.arange(1,12+1)
chunk_size = {'latitude':720,'longitude':1440}

# data analysis 
resample daily to monthly data

In [4]:
%%time
indir = ('/ihesp/user/dapengli/obs_dat/SEALEVEL_GLO_PHY_L4_REP_OBSERVATIONS_008_047/'
         'dataset_duacs_rep_global_merged_allsat_phy_l4/')
outdir = ('/scratch/user/dapengli/Projects4iHESP/Project_SeaLevelUSEastCoast_2021Mar30/'
          'data_after_manipulation/ADT_Vgeo_CMEMS/')

for iyr in yrs:
    for imon in mons:
        infiles = indir + str(iyr) + '/' + str(imon).zfill(2) + '/*.nc'
        ds = xr.open_mfdataset(infiles, compat="override", combine="by_coords", 
             data_vars="minimal", coords="minimal",  chunks=chunk_size, parallel=True)
        print('av.'+str(iyr)+str(imon).zfill(2)+', total days:'+str(ds.time.size))
        ds_rs = ds.resample(time='M').mean('time') # time='M', month end time
        
        outfile = (outdir + 'dt_global_allsat_phy_l4_' + str(iyr) + str(imon).zfill(2)
                   +'_'+date_str+'.nc')
        ds_rs.to_netcdf(path=outfile, mode='w', format='NETCDF4', compute=True)
        del infiles, ds, outfile, ds_rs

av.199301, total days:31
av.199302, total days:28
av.199303, total days:31
av.199304, total days:30
av.199305, total days:31
av.199306, total days:30
av.199307, total days:31
av.199308, total days:31
av.199309, total days:30
av.199310, total days:31
av.199311, total days:30
av.199312, total days:31
av.199401, total days:31
av.199402, total days:28
av.199403, total days:31
av.199404, total days:30
av.199405, total days:31
av.199406, total days:30
av.199407, total days:31
av.199408, total days:31
av.199409, total days:30
av.199410, total days:31
av.199411, total days:30
av.199412, total days:31
av.199501, total days:31
av.199502, total days:28
av.199503, total days:31
av.199504, total days:30
av.199505, total days:31
av.199506, total days:30
av.199507, total days:31
av.199508, total days:31
av.199509, total days:30
av.199510, total days:31
av.199511, total days:30
av.199512, total days:31
av.199601, total days:31
av.199602, total days:29
av.199603, total days:31
av.199604, total days:30


resample monthly to yearly data

In [4]:
outdir = ('/scratch/user/dapengli/Projects4iHESP/Project_SeaLevelUSEastCoast_2021Mar30/'
          'data_after_manipulation/ADT_Vgeo_CMEMS/')
infiles = [outdir + 'dt_global_allsat_phy_l4_' + str(iyr) + str(imon).zfill(2) + 
           '_2021May18.nc' for iyr, imon in product(yrs, mons)]
outfile = (outdir + 'dt_global_allsat_phy_l4_1993-2019_'+date_str+'.nc') # yearly av file
outfile2 = (outdir + 'dt_global_allsat_phy_l4_199301-201912_'+date_str+'.nc') # monthly av file

In [5]:
ds = xr.open_mfdataset(infiles, compat="override", combine="by_coords", 
             data_vars="minimal", coords="minimal",  chunks=chunk_size, parallel=True)
ds

<xarray.Dataset>
Dimensions:    (latitude: 720, longitude: 1440, nv: 2, time: 324)
Coordinates:
  * time       (time) datetime64[ns] 1993-01-31 1993-02-28 ... 2019-12-31
  * latitude   (latitude) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * longitude  (longitude) float32 0.125 0.375 0.625 0.875 ... 359.4 359.6 359.9
  * nv         (nv) int32 0 1
Data variables:
    crs        (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    lat_bnds   (time, latitude, nv) float32 dask.array<chunksize=(1, 720, 2), meta=np.ndarray>
    lon_bnds   (time, longitude, nv) float32 dask.array<chunksize=(1, 1440, 2), meta=np.ndarray>
    err        (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    adt        (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    ugos       (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    vgos       (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sla        (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    ugosa      (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    vgosa      (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>

In [6]:
# save monthly av file 
ds.to_netcdf(path=outfile2, mode='w', format='NETCDF4', compute=True)

In [6]:
# save yearly av file
ds_rs = ds.resample(time='A').mean('time') # time='A' year end 
ds_rs.to_netcdf(path=outfile, mode='w', format='NETCDF4', compute=True)
ds_rs

<xarray.Dataset>
Dimensions:    (latitude: 720, longitude: 1440, nv: 2, time: 27)
Coordinates:
  * time       (time) datetime64[ns] 1993-12-31 1994-12-31 ... 2019-12-31
  * latitude   (latitude) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * longitude  (longitude) float32 0.125 0.375 0.625 0.875 ... 359.4 359.6 359.9
  * nv         (nv) int32 0 1
Data variables:
    crs        (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
    lat_bnds   (time, latitude, nv) float32 dask.array<chunksize=(1, 720, 2), meta=np.ndarray>
    lon_bnds   (time, longitude, nv) float32 dask.array<chunksize=(1, 1440, 2), meta=np.ndarray>
    err        (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    adt        (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    ugos       (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    vgos       (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sla        (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    ugosa      (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    vgosa      (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>

# Appendix

In [4]:
iyr=1993
imon=12
infiles = indir + str(iyr) + '/' + str(imon).zfill(2) + '/*.nc'
infiles

'/ihesp/user/dapengli/obs_dat/SEALEVEL_GLO_PHY_L4_REP_OBSERVATIONS_008_047/dataset_duacs_rep_global_merged_allsat_phy_l4/1993/12/*.nc'

In [5]:
outfile=(outdir+'dt_global_allsat_phy_l4_'+str(iyr)+str(imon).zfill(2)+'_'+date_str+'.nc')
outfile

'/scratch/user/dapengli/Projects4iHESP/Project_SeaLevelUSEastCoast_2021Mar30/data_after_manipulation/ADT_Vgeo_CMEMS/dt_global_allsat_phy_l4_199312_2021May18.nc'

In [6]:
ds = xr.open_mfdataset(infiles, compat="override", combine="by_coords", data_vars="minimal",
                          coords="minimal",  chunks=chunk_size, parallel=True)
print('av.'+str(iyr)+'-'+str(imon).zfill(2)+', total days:'+str(ds.time.size))
ds

av.1993-12, total days:31


<xarray.Dataset>
Dimensions:    (latitude: 720, longitude: 1440, nv: 2, time: 31)
Coordinates:
  * time       (time) datetime64[ns] 1993-12-01 1993-12-02 ... 1993-12-31
  * latitude   (latitude) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * longitude  (longitude) float32 0.125 0.375 0.625 0.875 ... 359.4 359.6 359.9
  * nv         (nv) int32 0 1
Data variables:
    crs        int32 ...
    lat_bnds   (latitude, nv) float32 dask.array<chunksize=(720, 2), meta=np.ndarray>
    lon_bnds   (longitude, nv) float32 dask.array<chunksize=(1440, 2), meta=np.ndarray>
    err        (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    adt        (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    ugos       (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    vgos       (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sla        (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    ugosa      (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    vgosa      (time, latitude, longitude) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
Attributes: (12/44)
    Conventions:                     CF-1.6
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    cdm_data_type:                   Grid
    comment:                         Sea Surface Height measured by Altimetry...
    contact:                         servicedesk.cmems@mercator-ocean.eu
    creator_email:                   servicedesk.cmems@mercator-ocean.eu
    ...                              ...
    summary:                         SSALTO/DUACS Delayed-Time Level-4 sea su...
    time_coverage_duration:          P1D
    time_coverage_end:               1993-12-01T00:00:00Z
    time_coverage_resolution:        P1D
    time_coverage_start:             1993-12-01T00:00:00Z
    title:                           DT merged all satellites Global Ocean Gr...